In [2]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Make Detections

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])

    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

In [5]:
def calculate_statistics(landmarks, world_landmarks):
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    left_shoulder = (world_landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                 world_landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
    world_left_ankle = [world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
              world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
              world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
    world_left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
    right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
    left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
    right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    right_shoulder = (world_landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                  world_landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
    world_right_ankle = [world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
               world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
               world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]
    world_right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    
    # Hips Below knees
    if landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y <= landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y and landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y <= landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y:
        hipsBelowKnees = True
    else:
        hipsBelowKnees = False

    # Weight shift (Forward/Backward)
    shoulder_mid_x = (left_shoulder[0] + right_shoulder[0]) / 2
    ankle_mid_x = (left_ankle[0] + right_ankle[0]) / 2
    
    # Weight shift based on angle between toe heel ankle
    right_foot_angle = calculate_angle(right_foot_index, right_heel, right_ankle)
    left_foot_angle = calculate_angle(left_foot_index, left_heel, left_ankle)
    
    # Weight shift lateral
    left_mid_x = (left_hip[0] + left_ankle[0]) / 2

    metrics = {
    'depth_left': calculate_angle(left_ankle, left_knee, left_hip),
    
    'depth_right': calculate_angle(right_ankle, right_knee, right_hip),
    
    'knee_balance': [right_foot_angle, left_foot_angle],
    
    'hips_below_knees': hipsBelowKnees,
        
    'foot_distance': np.linalg.norm(np.array(world_left_ankle) - np.array(world_right_ankle)),
    
    'grip_width': np.linalg.norm(np.array(world_left_wrist) - np.array(world_right_wrist)),
    
    'weight_shift': (shoulder_mid_x - ankle_mid_x),
    
    'lateral_hip_position': left_mid_x,
    
    'spine_angle_1': calculate_angle(left_knee, left_hip, left_shoulder),
    'spine_angle_2': calculate_angle(right_knee, right_hip, right_shoulder),
    'spine_angle_3': left_shoulder[1] - left_hip[1]
    }
    
    return metrics


In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

counter = 0
stage = None
apex = 1000
prev_landmarks = None
prev_world_landmarks = None
top_rep_statistics = None
top_hip_position = 0
lateral_shift = 0
perSquatMetrics = {}
current_rep = {
    'max_depth': 0,
    'min_spine_angle': float('inf'),
    'max_lateral_shift': 0,
    'max_forward_shift': 0,
    'foot_distance': 0,
    'grip_width': 0,
    'knee_balance_bottom': None,
    'bottom_position_held': 0  # frames spent at bottom position
}

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Image because we want our image to be passed to MediaPipe in format to RGB (Default is of BGR)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection (Pose model)
        results = pose.process(image)
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            world_landmarks = results.pose_world_landmarks.landmark
            metrics = calculate_statistics(landmarks, world_landmarks)
            apex = min((world_landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y + world_landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y) / 2, apex) # Bottom of Squat
            
            # Display the metrics on the image
            print(metrics["knee_balance"])
            print(metrics["foot_distance"])
            cv2.putText(image, str(metrics["depth_left"]), (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(metrics["spine_angle_1"]), (10, 50), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, stage, (10, 70), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (10, 90), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Get Squat Metrics Per Frame and Per Rep
            if metrics["depth_left"] > 160 :
                if stage != "top rep":
                    if counter > 0:
                        perSquatMetrics[counter] = current_rep.copy()
                        current_rep = {
                            'max_depth': float('inf'),
                            'min_spine_angle': float('inf'),
                            'max_lateral_shift': 0,
                            'max_forward_shift': 0,
                            'foot_distance': 0,
                            'grip_width': 0,
                            'knee_balance_bottom': None,
                            'bottom_position_held': 0,  # frames spent at bottom position
                            'hips_below_knees': False
                        }
                stage = "top rep"
                top_hip_position = metrics["lateral_hip_position"]
                lateral_shift = 0
            if metrics["depth_left"] < 160 and stage == 'top rep':
                stage = "bottom rep"
                counter += 1
            if stage == "bottom rep":
                current_rep['max_depth'] = min(current_rep['max_depth'], metrics["depth_left"])
                current_rep['min_spine_angle'] = min(current_rep['min_spine_angle'], metrics["spine_angle_3"])
                lateral_shift = (top_hip_position - metrics["lateral_hip_position"]) * 100
                current_rep['max_lateral_shift'] = lateral_shift if abs(lateral_shift) > abs(current_rep['max_lateral_shift']) else current_rep['max_lateral_shift']
                current_rep['max_forward_shift'] = metrics['weight_shift'] if abs(metrics['weight_shift']) > abs(current_rep['max_forward_shift']) else current_rep['max_forward_shift']
                current_rep['foot_distance'] = metrics["foot_distance"]
                current_rep['grip_width'] = metrics["grip_width"]
                
                if metrics["hips_below_knees"]:
                    current_rep['hips_below_knees'] = True
                    
                # Track knee balance at bottom position
                if metrics["depth_left"] < 100:  # Deep squat position
                    current_rep['bottom_position_held'] += 1
                    if current_rep['knee_balance_bottom'] is None:
                        current_rep['knee_balance_bottom'] = metrics["knee_balance"]
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2))
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
        prev_landmarks = landmarks
        prev_world_landmarks = prev_world_landmarks

    cap.release()
    cv2.destroyAllWindows()
    
    

I0000 00:00:1740247893.449387 84893243 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1740247893.526563 84908503 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740247893.539470 84908505 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


[161.66646524064126, 142.37001857567247]
0.34636630143126496
[170.55637301810444, 140.10569578109713]
0.31804953059395574
[158.24148927254524, 134.31638525309623]
0.3098690905631904
[137.39932225022693, 109.4139547633016]
0.27094192820701485
[133.37537895802413, 74.93149677689584]
0.2379462491590197
[129.34745271312943, 39.08252003270514]
0.22298578094383922
[129.959844770766, 20.142119281361982]
0.2103601079481795
[125.9044477625246, 25.973781702116213]
0.22891752588675826
[135.90663898727254, 24.690294015859944]
0.2017405828329149
[145.61652104388867, 43.829975949085366]
0.21097677399984707
[140.2985358608417, 34.085510994454566]
0.20596096367272582
[127.4057877617185, 10.723987017228819]
0.19996580165614178
[142.76570339741184, 23.03731420379836]
0.20023967991386354
[41.48285320447491, 11.506042939201278]
0.21763072212414217
[150.20138074998908, 139.7262169716491]
0.2291218467050588
[137.96744855390384, 102.97040972970478]
0.236085044197831
[152.02803059212482, 123.20121470732562]
0

In [169]:
video = "backsquat.mp4"
cap = cv2.VideoCapture(video)

counter = 0
stage = None
apex = 1000
prev_landmarks = None
prev_world_landmarks = None
top_rep_statistics = None
top_hip_position = 0
lateral_shift = 0

with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:  # Prevent crash if frame is None
            break
        
        # Recolor Image because we want our image to be passed to MediaPipe in format to RGB (Default is of BGR)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection (Pose model)
        results = pose.process(image)
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            world_landmarks = results.pose_world_landmarks.landmark
            metrics = calculate_statistics(landmarks, world_landmarks)
            apex = min((world_landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y + world_landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y) / 2, apex) # Bottom of Squat
            
            # Display the metrics on the image
            cv2.putText(image, str(metrics["hips_below_knees"]), (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(metrics["spine_angle_1"]), (10, 50), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, (10, 70), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (10, 90), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            if metrics["depth_left"] > 170:
                stage = "top rep"
                top_hip_position = metrics["lateral_hip_position"]
                lateral_shift = 0
            if metrics["depth_left"] < 120 and stage == "top rep":
                stage = "during rep"
                counter += 1
            if metrics["depth_left"] < 150:
                lateral_shift = (top_hip_position - metrics["lateral_hip_position"]) * 100
                
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2))
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
        prev_landmarks = landmarks
        prev_world_landmarks = prev_world_landmarks

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1740244740.558183 84292171 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2 Pro
W0000 00:00:1740244740.647496 84851123 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740244740.659916 84851118 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


: 

In [9]:
print("\nSquat Analysis:")
for rep_num, metrics in perSquatMetrics.items():
    print(f"\nRep {rep_num}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")


Squat Analysis:

Rep 1:
max_depth: 167.35479478608863
min_spine_angle: -1.7143282294273376
max_lateral_shift: 33.35002064704895
max_forward_shift: -0.6501871161162853
foot_distance: 0.1852674833658955
grip_width: 0.7506196791623312
knee_balance_bottom: [47.06791293773782, 42.208244595435175]
bottom_position_held: 13
hips_below_knees: True

Rep 2:
max_depth: 68.89583309693239
min_spine_angle: -1.6275239288806915
max_lateral_shift: -7.175543904304504
max_forward_shift: -0.6663286038674414
foot_distance: 0.10449223110694952
grip_width: 0.46486560663789045
knee_balance_bottom: [17.56647900722913, 30.82126994473916]
bottom_position_held: 4
hips_below_knees: True

Rep 3:
max_depth: 157.49970849428007
min_spine_angle: 0.1395474374294281
max_lateral_shift: -9.461823105812073
max_forward_shift: -0.5552482940256596
foot_distance: 0.1488399162694974
grip_width: 0.3111586456609195
knee_balance_bottom: None
bottom_position_held: 0
hips_below_knees: False

Rep 4:
max_depth: 166.5789909168401
min_sp